# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# Импорт библиотек

In [1]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

import numpy as np

# Загрузка данных

**Описание данных**

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.

Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [2]:
df = pd.read_csv('insurance.csv')
print(df.info())
print(df.describe())
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None
               Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
count  5000.000000  5000.000000   5000.000000  5000.000000        5000.000000
mean      0.499000    30.952800  39916.360000     1.194200           0.148000
std       0.500049     8.440807   9900.083569     1.091387           0.463183
min       0.000000    18.000000   5300.000000     0.000000           0.000000
25%       0.000000    24.000000  33300.000000     0.000000           0.000000
50%       0.000000    30.000000  

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


Пропусков нет, откровенных выбросов нет.

In [3]:
# Разбивка на признаки и целевой признак
X = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']

# Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $w_{ciphered}$ — вектор весов линейной регрессии после шифровки (нулевой элемент равен сдвигу)

- $a$ — вектор предсказания

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

**Ответ:** Нет.

**Обоснование:** Поскольку качество линейной регрессии в данной задаче измеряется метрикой R2, то необходимо построить две модели линейной регрессии в одну дать оригинальные данные, в другую зашифрованные и сравнить их метрики R2 на тестовой выборке.

**Алгебраическое доказательство:**

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$
$$
w' = P^{-1} w
$$

**Практическое доказательство:**

Поскольку из матрицы признаков необходимо после умножения получить матрицу того же размера что и оригинальная матрица, то будем умножать на квадратную матрицу размером 4х4 (у нас 4 признака).

Создадим случайную обратимую матрицу 4х4.

In [4]:
rng = np.random.default_rng(24)
key = rng.normal(0, 1, size=(4, 4))
while True:
    if np.linalg.det(key) == 0: # Проверка на обратимость
        key = rng.normal(0, 1, size=(4, 4))
    else: break

С помощью перемножения на созданную случайную матрицу зашифруем данные.

In [5]:
X_ciphered = pd.DataFrame(X @ key)
X_ciphered.columns = X.columns

Разобъем датафрейм на обучающую и тестовую выборки.

In [6]:
X_train, X_test, X_ciphered_train, X_ciphered_test, y_train, y_test = train_test_split(X, X_ciphered, y, test_size=0.2, random_state=24)

И прогоним через модели.

In [7]:
pipe = Pipeline([ # Тут можно было просто подствить все в LinearRegression модель, но я раньше не пользовался пайплайном и мне хотелось попробовать.
    ('scaler', StandardScaler()), # Не влияет на полученные метрики
    ('model', LinearRegression())
])

pipe.fit(X_train, y_train) # Предсказание по оригинальным данным
R2 = pipe.score(X_test, y_test)
predicted = pipe.predict(X_test)

pipe.fit(X_ciphered_train, y_train) # Предсказание по шифрованным данным
R2_ciphered = pipe.score(X_ciphered_test, y_test)
predicted_ciphered = pipe.predict(X_ciphered_test)

print(f'''
R2 для оригинальных данных = {R2}, R2 для зашифрованных данных = {R2_ciphered}, разница между ними = {R2 - R2_ciphered}
MAE между предсказаниями на оригинальных данных и предсказаниями на шифрованных данных = {mean_absolute_error(predicted, predicted_ciphered)}
''')


R2 для оригинальных данных = 0.37533999845221444, R2 для зашифрованных данных = 0.375339998452203, разница между ними = 1.1435297153639112e-14
MAE между предсказаниями на оригинальных данных и предсказаниями на шифрованных данных = 1.2380603486850107e-13



Судя по крайне низкой разнице между метриками R2, и по более чем скромной величине MAE эти модели очень близки друг другу по качеству.

---

Из любопытства по формуле $w = (X^T X)^{-1} X^T y$ проверим как поменяются коэффициенты будущей линейной регрессии.

In [8]:
w = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
w_ciphered = np.linalg.inv(X_ciphered_train.T @ X_ciphered_train) @ X_ciphered_train.T @ y_train
print(f'w = {w.values}, w после шифровки = {w_ciphered.values}')

w = [-4.62638950e-02  2.40537650e-02 -1.21177835e-05 -4.70494952e-02], w после шифровки = [-0.04557396 -0.04312553 -0.02421157 -0.01033225]


Параметры линейной регрессии в исходной задаче и в преобразованной связаны следующим образом:

Поскольку $a = XPw_{ciphered}$ и $a = Xw$, то их можно привести к одному уравнению: $XPw_{ciphered} = Xw$, которое после преобразования примет вид: $w_{ciphered} = P^{-1}w$

Проверим полученную формулу с помошью рассчета:

In [9]:
w_check = np.linalg.inv(key) @ w
print(f'w при расчете через альтернативную формулу = {w_check}')

w при расчете через альтернативную формулу = [-0.04557388 -0.04312548 -0.02421157 -0.01033228]


Что примерно совпадает с ранее полученным результатом.

# Алгоритм преобразования

**Алгоритм**

...

**Обоснование**

...

Алгоритм шифровки и его обоснование указаны выше, в пункте **Умножение матриц**.

Но можно отметить, что для постоянства шифровки в качестве "пароля" можно применять seed, забрасываемый в код: `rng = np.random.default_rng(fill_me_with_your_seed)`. В моем случае пароль это число 24.

# Проверка алгоритма

Алгоритм написан и проверен выше, в пункте **Умножение матриц**.